# Instructions:
- Check the instructions in the README.md in the root of this repository
- Run all code in sequence
- :D

In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import glob
import os
import requests
import openpyxl  # TODO: is this used?

In [2]:
debug_mode = False

# Loading All the HTML files

In [3]:
# Create directory to store the html files
os.makedirs('html', exist_ok=True)
# Use glob to find all the html files in the html directory
html_files = glob.glob('google_html/*.html')

# Obtain all company names, their website URLs, and GPS coordinates, from the html files

In [4]:
for html_file in html_files:
    # Load html content from a file "IT companies south east Drenthe.html"
    with open(html_file, encoding='utf-8') as file:
        html_content = file.read()

        # Create a BeautifulSoup object
        soup = BeautifulSoup(html_content, 'html.parser')

        # Find all parent elements that contain both the title and the URL
        parent_elements = soup.find_all("div", class_="qBF1Pd fontHeadlineSmall")

        company_names = []  # These are named titles in Google Maps jargon
        company_urls = []
        seen_urls = {}  # There was a bug in the code that caused duplicate URLs to be added to the list, so we need to keep track of the URLs we have already seen and filter those out. Not the pretiest solution, but it works.
        for parent in parent_elements:
            title = re.sub(' +', ' ', parent.get_text().replace('\n', '').strip())
            link_tag = parent.find_next("a", class_="lcr4fd S9kvJb")
            url = link_tag['href'] if link_tag and 'href' in link_tag.attrs else None
            if url in seen_urls:
                company_urls[seen_urls[url]] = ""
            seen_urls[url] = len(company_urls)
            company_names.append(title)
            company_urls.append(url)

        if debug_mode: print("Company names:", company_names)
        if debug_mode: print("Company URLs:", company_urls)

        # Find the href in the a tag with class "hfpxzc"
        google_urls = [link['href'] for link in soup.find_all("a", class_="hfpxzc")]
        if debug_mode: print("Google URLs:", google_urls)

        # Use regular expressions to find latitude and longitude
        latitude = [re.search(r'!3d([-.\d]+)', url) for url in google_urls]
        longitude = [re.search(r'!4d([-.\d]+)', url) for url in google_urls]

        # Extract and convert to float
        latitude = [float(lat.group(1)) if lat else None for lat in latitude]
        longitude = [float(lon.group(1)) if lon else None for lon in longitude]

        # Print the GPS coordinates, for debugging purposes
        gps_coordinates = [(lat, long) for lat, long in zip(latitude, longitude)]
        if debug_mode: print("GPS Coordinates:", gps_coordinates)

        # Put the data into a pandas DataFrame
        df = pd.DataFrame({'Company name': company_names, 'Company URL': company_urls, 'Google URL': google_urls, 'Latitude': latitude, 'Longitude': longitude})
        if debug_mode: print(df)

        # Create directory to store a csv file per html file
        os.makedirs('csv', exist_ok=True)

        # Save the output to a csv file in the csv directory, using the name of the html file
        csv_file = os.path.basename(html_file).replace('.html', '.csv')
        df.to_csv(os.path.join('csv', csv_file), index=False)

# Visit all Company URLs, get frontpage html, extract just the text, and ask ChatGPT about a company summary

In [ ]:
def extract_text_from_html(html_content):
    """
    Extracts all human-readable text from an HTML document.
    """
    soup = BeautifulSoup(html_content, "html.parser")

    # Remove script and style elements
    for tag in soup(["script", "style", "noscript", "meta", "link", "head"]):
        tag.extract()

    # Get text and clean up unnecessary whitespace
    text = soup.get_text(separator=" ", strip=True)

    return text

import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"): # Andrew mentioned that the prompt/ completion paradigm is preferable for this class
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
import glob
import csv
import requests
from bs4 import BeautifulSoup

def extract_text_from_html(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup.get_text(separator=' ', strip=True)

def process_csv_file(csv_file):
    rows = []
    
    with open(csv_file, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        fieldnames = reader.fieldnames + ["Description", "ChatGPT Description", "ChatGPT Keywords"]
        
        for row in reader:
            url = row.get("Company URL", "").strip()
            
            if url:
                try:
                    response = requests.get(url)
                    if response.status_code == 200:
                        html_content = response.text
                        soup = BeautifulSoup(html_content, 'html.parser')
                        frontpage_text = extract_text_from_html(html_content)

                        prompt_desc = f"The text of the frontpage of a company will be provided, delimited by triple backticks. Please provide a one-sentence summary of the company. ```{frontpage_text}```"
                        chatgpt_description = get_completion(prompt_desc)

                        prompt_keywords = f"The text of the frontpage of a company will be provided, delimited by triple backticks. Please provide five keywords that describe the company. ```{frontpage_text}```"
                        chatgpt_keywords = get_completion(prompt_keywords)

                        meta_description = soup.find("meta", attrs={"name": "description"})
                        html_description = meta_description['content'] if meta_description else None
                    else:
                        html_description = chatgpt_description = chatgpt_keywords = None
                except requests.RequestException:
                    html_description = chatgpt_description = chatgpt_keywords = None
            else:
                html_description = chatgpt_description = chatgpt_keywords = None
            
            row["Description"] = html_description
            row["ChatGPT Description"] = chatgpt_description
            row["ChatGPT Keywords"] = chatgpt_keywords
            rows.append(row)
    
    with open(csv_file, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

# Load all CSV files in the 'csv' directory
csv_files = glob.glob('csv/*.csv')

for csv_file in csv_files:
    process_csv_file(csv_file)


# Extract text from the downloaded html frontpages, put them in csvs

In [ ]:
htmls = glob.glob("html/*/*.html")


# List to store extracted texts
data = []

# Loop through HTML files and extract text
for html in htmls:
    with open(html, "r", encoding="utf-8") as file:
        html_content = file.read()
    extracted_text = extract_text_from_html(html_content).replace("\n", " ").replace("\r", " ")
    
    # Append to data list
    data.append({"Filename": os.path.basename(html), "ExtractedText": extracted_text})

# Convert list to DataFrame
df = pd.DataFrame(data)

# Save DataFrame to CSV
df.to_csv("extracted_texts.csv", index=False, encoding="utf-8")

In [39]:
# # open extracted_texts.csv and read the content
# df = pd.read_csv("extracted_texts.csv")
# df.head()

import csv

with open("extracted_texts.csv", mode='r', newline='', encoding='utf-8') as file:
    reader = csv.reader(file)  # Create a CSV reader object
    print("[", end="")  # Print the start of the list
    for row in reader:  # Iterate over rows
        print(row, ",", end="")  # Print each row as a list
    print("]")


[['Filename', 'ExtractedText'] ,['IoT Nederland.html', 'IoT Nederland Home Hoe werkt het Wat doen wij Contact Een wereld van ongekende mogelijkheden Internet of Things Een golf van "Smart-devices" gaat voor een ware revolutie zorgen in              uw huis, in uw auto, op uw werk, in uw bedrijf maar wellicht ook              in uw producten en uw businessmodel. Hoe Smart kunnen wij uw omgeving en uw product maken? Intelligente apparaten komen.. geen twijfel mogelijk. Tot nu toe leefde Internet in een klein doosje.. een PC, laptop,            tablet, in uw broekzak of tas. Nu komt internet uit het doosje en            manifesteert zich in de vorm van smart-devices overal om ons heen. Bij            "Smart-devices" denken mensen meestal aan hun "Smart-televisie" of hun            "Smartphone" maar binnen 5 jaar zal uw huis, uw auto, uw werkplek            overspoeld worden door smart-devices die U allerlei taken en            beslissingen uit handen zullen nemen of u zullen roepen wannee

# Convert the CSVs to one Excel

In [22]:
# Load all csv files in the csv directory
csv_files = glob.glob('csv/*.csv')

# If it exist open company_summary.csv
if os.path.exists('company_summary.csv'):
    # Read the CSV file into a DataFrame
    company_summaries = pd.read_csv('company_summary.csv')


# Create an Excel file, that will contain all csv files, each in a separate sheet
with pd.ExcelWriter('Companies.xlsx') as writer:
    for csv in csv_files:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv)
        # Get the name of the CSV file (without the directory and extension)
        sheet_name = os.path.splitext(os.path.basename(csv))[0]
        # Write the DataFrame to the Excel file
        df.to_excel(writer, sheet_name=sheet_name, index=False)


IndexError: At least one sheet must be visible